**CAI A Deep Dive into ChatGPT Functions for Enhanced UX**

Supporting code for the article: ** medium article URL **

Jose Valdivia Leon
j.leon@evoai.onmicrosoft.com
[LinkedIn Profile](https://www.linkedin.com/in/jose-valdivia-leon-056145/)

In [1]:
import openai
import json
import os

# load and set our key
openai.api_key = os.environ.get('OPENAI_API_KEY')

Completion with Functions: To do this, we want to make sure that we're using version `gpt-3.5-turbo-0613` or later. 

In [2]:
function_legs = [
        {
            "name": "get_flight_legs",
            "description": "Get a list representing the flight departure and arrival cities and the departure date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "legs": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "departure_city": {
                                    "type": "string",
                                    "description": "the departure city of the flight leg."
                                },
                                "arrival_city": {
                                    "type": "string",
                                    "description": "the arrival city of the flight leg"
                                },
                                "departure_date": {
                                    "type": "string",
                                    "format": "date",
                                    "description": "the departure date of the flight leg"
                                }
                            },
                            "description": "List of departure and arrival cities and the departure date."
                        }
                    }
                },
                "required": [
                    "legs"
                ]
            }
        }
    ]


In [3]:
system_input = "Think step by step, check if you have all cities and all dates needed. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous or you need more data. Consider that the current date and time is: Fri Jun 30 2023 11:27:39 GMT+0100 (British Summer Time)."
user_input = "I'm flying from Cologne to London next Monday, stay in London for 5 days and the fly to Rome for a week, then back to Cologne."
functions = function_legs

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
        ],
    functions=functions,
    function_call={"name": "get_flight_legs"},
    temperature=0
)

reply_content = completion.choices[0]
reply_content

<OpenAIObject at 0x21bc955cd70> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_flight_legs",
      "arguments": "{\n  \"legs\": [\n    {\n      \"departure_city\": \"Cologne\",\n      \"arrival_city\": \"London\",\n      \"departure_date\": \"2023-07-03\"\n    },\n    {\n      \"departure_city\": \"London\",\n      \"arrival_city\": \"Rome\",\n      \"departure_date\": \"2023-07-10\"\n    },\n    {\n      \"departure_city\": \"Rome\",\n      \"arrival_city\": \"Cologne\",\n      \"departure_date\": \"2023-07-17\"\n    }\n  ]\n}"
    }
  },
  "finish_reason": "stop"
}

We can convert this OpenAI object to a more familiar Python dict by doing:

In [4]:
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
print(funcs)
print(funcs['legs'])


{'legs': [{'departure_city': 'Cologne', 'arrival_city': 'London', 'departure_date': '2023-07-03'}, {'departure_city': 'London', 'arrival_city': 'Rome', 'departure_date': '2023-07-10'}, {'departure_city': 'Rome', 'arrival_city': 'Cologne', 'departure_date': '2023-07-17'}]}
[{'departure_city': 'Cologne', 'arrival_city': 'London', 'departure_date': '2023-07-03'}, {'departure_city': 'London', 'arrival_city': 'Rome', 'departure_date': '2023-07-10'}, {'departure_city': 'Rome', 'arrival_city': 'Cologne', 'departure_date': '2023-07-17'}]
